# Imports

In [53]:
import requests
from flask import jsonify
import json
from datetime import date, time, datetime
from seleniumrequests import Firefox, PhantomJS
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
import pdfminer.layout
import pdfminer.high_level
import pdfplumber
import pdfminer.pdftypes
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextBox
from pdfminer.pdfpage import PDFPage
import ssl
import certifi
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfFileWriter
import os
import requests


# Testing POST /merge_pdfs endpoint

In [5]:
body_all = {"1": {"bp_bank_id": "0170",
             "last_updated": "2021-04-06",
             "url": "https://www.abanca.pt"
             },
             "2": {"bp_bank_id": "0269",
             "last_updated": "2021-04-06",
             "url": "https://www.bankinter.pt"
             },
             "3": {"bp_bank_id": "0079",
             "last_updated": "2021-04-06",
             "url": "https://www.bancobic.ao"
             },
             "4": {"bp_bank_id": "0008",
             "last_updated": "2021-04-06",
             "url": "https://www.bancobai.ao"
             },
             "5": {"bp_bank_id": "0193",
             "last_updated": "2021-04-06",
             "url": "https://www.bancoctt.pt/"
             }
            }

In [23]:
# requires this request body
body = {"3": {"bp_bank_id": "0079",
"last_updated": "2021-04-06",
"url": "https://www.eurobic.pt/"
}
}
headers = {'User-Agent':'Chrome/50.0.2661.102'}

In [28]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/merge_pdfs"
url_h = 'https://bank-price-api.herokuapp.com/merge_pdfs'

In [29]:
response = requests.post(url, data = None, json = body, headers=headers, verify = False).json()

In [18]:
ident = response.get('ident')

In [7]:
ident

636

## Testing GET /retrievepdfs

Since the ```/merge_pdfs``` job can take some time and we want to avoid heroku timeout issues the job is done in the background. To pick up the requested file, the rails app can go to this endpoint after some waiting time. 
There will be only one required argument ```ident``` to be passed to the GET request which was given before by the response of ```/merge_pdfs```.

In [8]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/retrievepdfs"
url_h = 'https://bank-price-api.herokuapp.com/retrievepdfs'

In [15]:
params = {'ident':ident}


NameError: name 'ident' is not defined

In [10]:
# call this after logs say: 'updated bank.json ready for pickup with ident: <ident>'
response = requests.get(url, params=params).json()

In [11]:
response

{'message': "sourcing job not finished or initialized or ident: 636 is not available. first call /merge_pdfs and wait for backgroundjob to finish. Error msg: [Errno 2] No such file or directory: 'bank_benchmark_api/data/banks_636.json'",
 'status': 'error'}

In [12]:
# saving infos for the other endpoints
bp_pdf_url = response.get('1').get('bp_pdf_url')
bp_bank_id = response.get('1').get('bp_bank_id')

AttributeError: 'NoneType' object has no attribute 'get'

# Testing POST /get_stats

In [67]:
# requires this request body
body = {
  "1": {
    "url": "https://www.abanca.pt",
    "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf",
    "bp_bank_id": "0193",
    "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf",
    "products": {
      "demand_deposit": {
        "commissions": {
          "statement": [
            "Emissão de extrato",
            "Extrato Integrado",
            "Extrato Mensal",
            "Extrato integrado",
            "Extrato avulso"
          ],
          "documents_copy": [
            "Fotocópias de segundas vias de talões de depósito",
            "Emissão 2ªs Vias de Avisos e Outros Documentos",
            "Extracto avulso",
            "Segundas vias (pedido na agência)"
          ],
          "acc_manteinance": [
            "Manutenção de conta",
            "Comissão de manutenção de conta",
            "Comissão de Manutenção de Conta",
            "Manutenção de Conta Pacote",
            "Manutenção de Conta Base",
            "Manutenção de Conta Serviços Mínimos Bancários"
          ],
          "withdraw": [
            "Levantamento de numerário",
            "Levantamento de numerário ao balcão",
            "Comissão de Levantamento",
            "Levantamento de Numerário ao Balcão",
            "Levantamento de Numerário ao Balcão"
          ],
          "online_service": [
            "Adesão ao serviço de banca à distância",
            "Adesão ao serviço online"
          ],
          "cash_deposit": [
            "Depósito de moedas metálicas",
            "Depósito de moedas",
            " Depósito em moeda metálica (>= 100 moedas)",
            "Depósito de moedas ao balcão",
            "Depósito de dinheiro ao balcão",
            "Depósito em moeda metálica (>= 100 moedas)"
          ],
          "change_holder": [
            "Alteração de titulares",
            "Alteração de titularidade",
            "Comissão de Alteração de Titularidade",
            "Alteração de titularidade / intervenientes",
            "Alteração de titularidade (titular/ representante)"
          ],
          "bank_overdraft": [
            "Comissões por descoberto bancário",
            "Descoberto bancário",
            "Comissões por Descoberto Bancário"
          ],
          "movement_consultation": [
            "Consulta de Movimentos de conta DO com",
            "Consulta de movimentos ao balcão"
          ],
          "balance_inquiry": [
            "Pedido de saldo ao balcão",
            "Consulta de Saldo de conta DO com comprovativo"
          ]
        },
        "portuguese": "Contas de Depósito"
      },
      "housing_credit": {
        "commissions": {
          "admin": [
            "Comissões associadas a atos administrativos 4.1 Não realização da escritura",
            "Alteração do local da escritura",
            "Declarações de dívida",
            "Mudança de regime de crédito",
            "Declarações de dívida",
            "Pedido de 2ª via de Caderneta Predial",
            "Emissão de declarações não obrigatórias por lei",
            "Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente",
            "Emissão de 2º vias de Declaração para efeitos de IRS",
            "Emissão de 2ª vias de faturas",
            "Declaração de Dívida para Fins Diversos",
            "Declaração de Encargos com Prestações"
          ],
          "certificates": [
            "Emolumentos do registo predial",
            "registo predial",
            "Certidão permanente on-line"
          ],
          "debt_recovery": [
            "Comissão de recuperação de valores em dívida",
            "Prestação até 50.000 €",
            "Prestação > 50.000 €",
            "Comissão de recuperação de valores em dívida",
            "Prestação > 50.000,00€",
            "Prestação ≤ 50.000,00€"
          ],
          "displacement": [
            "Comissão de deslocação",
            "Até 100 Kms",
            "101 a 250 Kms",
            "> 250 Kms "
          ],
          "early_payment": [
            "Comissão de reembolso antecipado parcial",
            "Taxa fixa",
            "Taxa variável",
            "Taxa fixa",
            "Comissão de reembolso antecipado total",
            "Comissão de antecipação",
            "(pré.aviso 7 dias)",
            "Comissão de compra antecipada",
            "(pré-aviso 10 dias)",
            "Comissão de Reembolso Antecipado Parcial",
            "Comissão de reembolso antecipado total"
          ],
          "evaluation": [
            "Avaliação",
            "Imóvel residencial",
            "Garagens e arrecadações não anexas ao imóvel residencial",
            "Avaliação do Imóvel"
          ],
          "formalization": [
            "Comissão de formalização",
            "Formalização"
          ],
          "process": [
            "Processo",
            "Abertura de Processo",
            "Desistência ou não conclusão do processo por motivos imputáveis ao cliente"
          ],
          "inspections": [
            "Vistorias",
            "em caso de construção ou realização de obras"
          ],
          "reanalysis": [
            "Reanálise"
          ],
          "settlement": [
            "Comissão de Liquidação de Prestação",
            "Liquidação de Prestação"
          ],
          "solicitors_notary": [
            "Emolumentos notariais",
            "Solicitadoria",
            "Notiário"
          ],
          "statements": [
            "Emissão de extratos de conta de empréstimos liquidados",
            "extrato",
            "extratos",
            "extrato de conta",
            "extrato mensal"
          ],
          "taxes": [
            "Imposto do Selo sobre concessão de crédito",
            "imposto",
            "imposto de selo",
            "impostos"
          ],
          "termination": [
            "Cessação da posição contratual",
            "cessação",
            "rescisão",
            "encerramento"
          ]
        },
        "portuguese": "Crédito à habitação e outros créditos hipotecários"
      }
    }
  }
}


In [68]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/get_stats"
url_h = 'https://bank-price-api.herokuapp.com/get_stats'

In [75]:
response = requests.post(url, data = None, json = body).json()
response

{'ident': 9731, 'started': True, 'status': 'ok', 'thread_name': 'Thread-3'}

In [ ]:
ident_s = response.get('ident')

In [ ]:
ident_s

## Testing GET /retrievestats

Since the ```/get_stats``` job can take some time and we want to avoid heroku timeout issues the job is done in the background. To pick up the requested file, the rails app can go to this endpoint after some waiting time. 
There will be only one required argument ```ident``` to be passed to the GET request which was given before by the response of ```/get_stats```.

In [ ]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/retrievestats"
url_h = 'https://bank-price-api.herokuapp.com/retrievestats'

In [ ]:
params = {'ident':ident_s}

In [ ]:
# call this after logs say: 'bank price json with <ident> loaded, supplied and removed from server'
response = requests.get(url, params=params).json()

In [ ]:
response

# Special cases selenium

In [21]:
proxies = {
 "http": "http://185.51.92.108:51327",
 "https": "http://185.51.92.108:51327",
}
headers = {'Accept-Language': 'pt-PT'}

In [ ]:
## Bancobai doesnt allow automatized parsing: 
url = 'https://www.bancobaieuropa.pt'
# response = requests.get(url, headers=headers, verify=False)

In [ ]:
PHANTOMJS_PATH = '/opt/homebrew/bin/geckodriver'
driver = webdriver.Firefox()
driver.get(url)
driver.implicitly_wait(8)
body = driver.page_source

In [22]:
# let's parse our html

soup=BeautifulSoup(driver.page_source)

NameError: name 'driver' is not defined

In [23]:
search = ['preçário', 'pricelist', 'precario']
for link in soup.find_all('a', href=True):
    url_prices = str(link.get('href').lower().strip())
    lower = str(link.string).lower().strip()
    title = str(link.get('title')).strip().lower()
    searchstring = ' '.join([url_prices, lower, title])
    if any([x in searchstring for x in search]):
        print(f'found terms of {search} in string {searchstring}')

NameError: name 'soup' is not defined

In [24]:
soup

NameError: name 'soup' is not defined

In [25]:
## now pdfs...
pdfs = ['https://www.bancobaieuropa.pt/sites/default/files/2020-11/Servico_de_Mudanca_de_Conta_Bancaria_PT.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2020-11/Documento%2520de%2520Informa%25C3%25A7%25C3%25A3o%2520Sobre%2520Comiss%25C3%25B5es_PRT_com%2520declara%25C3%25A7%25C3%25A3o_v01102020.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2021-01/PRE-FC-20210125.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2021-04/PRE-FT-20210407.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2020-11/Gloss%25C3%25A1rio_Doc.%2520informa%25C3%25A7%25C3%25A3o%2520sobre%2520comiss%25C3%25B5es_PT%2520e%2520ENG.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2018-07/Servico_Mudanca_Conta_Bancaria.pdf', 'http://www.bancobaieuropa.pt/sites/default/files/2019-12/direitos-pagamentos-europa.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2018-03/direitos_e_deveres_dos_consumidores_credito.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2018-03/depositos_bancarios_direitos_deveres.pdf', 'https://www.bancobaieuropa.pt/sites/default/files/2021-03/ficha_%2520de_informacao_depositante.pdf']
response = requests.get(pdfs[2], headers=headers, verify=False)

In [26]:
def file_decrypt(pdf_url, filename="tempe.pdf"):
    """Accepts pdf urls. Returns PyPDF2 FileReader Objects"""
    print(f'file_decrypt was called for {pdf_url}')
    response = requests.get(pdf_url, verify=False)
    if response.status_code == 200:
        temp = open(filename, "wb")
        temp.write(response.content)
        ## important: have to close again
        temp.close()
        # copies the file in temp.pdf / decrypts it and replaces the old file
        command="cp "+filename+" temp.pdf; qpdf --password='' --decrypt temp.pdf "+filename+ "; rm temp.pdf"
        os.system(command)
        print('file decrypted (with qpdf)')
        #re-open the decrypted file
        pdfFile = PdfFileReader(filename)
        # removing tempe file
        os.remove(filename)
        return pdfFile
    else:
        print(f'{url} could not be reached with file_decrypt. Response: {response}')
        return None

In [30]:
gcontext = ssl.SSLContext()  # Only for gangstars
# urlopen('https://example.com/bar/baz.html', cafile=certifi.where())

count = []
merger = PdfFileMerger()
for n, pdf in enumerate(pdfs):   
    try: 
        remote = requests.get(pdf, verify=False).content
        memory = BytesIO(remote)
        pdf_file = PdfFileReader(memory)
        if pdf_file.isEncrypted:
            print('file is encrypted')
            pdf_file = file_decrypt(pdf)
        count.append(n)
        merger.append(pdf_file)
        
        
    except Exception as e: 
        print(f'skipping: {pdf}, error: {e}')
        
# path = , 'w')
merger.write('testpdf.pdf')
        

skipping: https://www.bancobaieuropa.pt/sites/default/files/2020-11/Documento%2520de%2520Informa%25C3%25A7%25C3%25A3o%2520Sobre%2520Comiss%25C3%25B5es_PRT_com%2520declara%25C3%25A7%25C3%25A3o_v01102020.pdf, error: EOF marker not found


file is encrypted
file_decrypt was called for https://www.bancobaieuropa.pt/sites/default/files/2021-01/PRE-FC-20210125.pdf


file decrypted (with qpdf)


file is encrypted
file_decrypt was called for https://www.bancobaieuropa.pt/sites/default/files/2021-04/PRE-FT-20210407.pdf


file decrypted (with qpdf)


skipping: https://www.bancobaieuropa.pt/sites/default/files/2020-11/Gloss%25C3%25A1rio_Doc.%2520informa%25C3%25A7%25C3%25A3o%2520sobre%2520comiss%25C3%25B5es_PT%2520e%2520ENG.pdf, error: EOF marker not found


file is encrypted
file_decrypt was called for https://www.bancobaieuropa.pt/sites/default/files/2018-03/depositos_bancarios_direitos_deveres.pdf


file decrypted (with qpdf)


skipping: https://www.bancobaieuropa.pt/sites/default/files/2021-03/ficha_%2520de_informacao_depositante.pdf, error: EOF marker not found


In [31]:
count

[0, 2, 3, 5, 6, 7, 8]